In [1]:
!python -m spacy download en_core_web_sm

^C
Traceback (most recent call last):
  File "/home/lschettini/.pyenv/versions/3.9.20/lib/python3.9/runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/home/lschettini/.pyenv/versions/3.9.20/lib/python3.9/runpy.py", line 147, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/home/lschettini/.pyenv/versions/3.9.20/lib/python3.9/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/home/lschettini/.pyenv/versions/3.9.20/envs/medbench-alignscore-39/lib/python3.9/site-packages/spacy/__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
  File "/home/lschettini/.pyenv/versions/3.9.20/envs/medbench-alignscore-39/lib/python3.9/site-packages/spacy/pipeline/__init__.py", line 2, in <module>
    from .dep_parser import DependencyParser
  File "spacy/pipeline/dep_parser.pyx", line 1, in init spacy.pipeline.dep_parser
  File "spacy/pipeline/transiti

In [9]:
!pip show torch

Name: torch
Version: 1.13.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/lschettini/.pyenv/versions/3.9.20/envs/medbench-alignscore-39/lib/python3.9/site-packages
Requires: nvidia-cublas-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, typing-extensions
Required-by: alignscore, pytorch-lightning, torchmetrics


In [2]:
import os

from alignscore import AlignScore

/home/lschettini/.pyenv/versions/3.9.20/envs/medbench-alignscore-39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import nltk

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
scorer = AlignScore(
    model="roberta-base",
    batch_size=32,
    device="cpu",
    ckpt_path=os.path.abspath("../data/models/alignscore/AlignScore-base.ckpt"),
    evaluation_mode="nli_sp",
)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../data/models/alignscore/AlignScore-base.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lschettini/.pyenv/versions/3.9.20/envs/medbench-alignscore-39/lib/python3.9/site-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in 

In [5]:
scorer_large = AlignScore(
    model="roberta-base",
    batch_size=32,
    device="cpu",
    ckpt_path=os.path.abspath("../data/models/alignscore/AlignScore-large.ckpt"),
    evaluation_mode="nli_sp",
)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Lightning automatically upgraded your loaded checkpoint from v1.8.0.post1 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../data/models/alignscore/AlignScore-large.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sanity check:

In [4]:
score = scorer.score(contexts=["hello world."], claims=["hello world."])

Evaluating: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


# Consistency and relevancy:

In [23]:
sentence_pairs = [
    # ("The patient is a 36 year old male presenting with CHD.", "The patient complains and denies any previous heart problems."),
    ("The patient is a 36 year old male presenting with Chronic Heart Disease.", "The patient denies any previous heart problems."),
    # ("* The patient is a 36 year old male presenting with CHD.", "The patient complains and denies any previous heart problems."),
    # ("- The patient is a 36 year old male presenting with CHD.", "The patient complains and denies any previous heart problems."),
    ("The patient is a 36 year old male presenting with Chronic Heart Disease.", "The patient confirms previous heart problems."),
    ("The patient is a 36 year old male presenting with Chronic Heart Disease.", "The patient complains of lower back pain after mild physical activity."),
    ("The patient is a 36 year old male presenting with Chronic Heart Disease.", "The patient presents with mild chest pain radiating to the left arm."),
    ("The patient suffers from shortness of breath after daily exercise.", "The vase was on the edge of the nightstand."),
    ("The patient suffers from shortness of breath after mild exercise.", "The patient reports from dyspnea after mild physical activity."),
]

In [26]:
sentence_pairs = [
    ("John is going to New York", "John is heading to New York"),
    ("John is going to New York", "John is going to Brooklyn"),
    ("John is going to New York", "John is coming from New York"),
    ("This is a fact that is happening now: John is going to New York", "This is a fact that is happening now: John is coming from New York"),
    ("John is going to New York", "John is not going to New York"),
    ("John is going to New York", "John is going to Boston"),
    ("John is going to New York", "Mary is going to New York"),
]

In [18]:
lower_back_sentences = [
    "Doctor, I woke up this morning with a sharp pain in my lower back that just won't go away.",
    "I've been lifting heavy boxes at work, and now my back hurts really bad.",
    "I don't know what's wrong, but my back is killing me, and I can hardly move.",
    "I've been experiencing some intense discomfort in my back since yesterday, and it's getting worse.",
    "My back feels really tight and painful, especially when I try to bend over or pick something up.",
    "Doc, I have this sudden, stabbing pain in my back that started after I did some heavy lifting.",
    "I can barely stand up straight because my back is in so much pain; it feels like a muscle spasm.",
    "I've never had back issues before, but now I can't sit or stand without feeling this awful pain.",
    "I think I might have hurt my back while exercising; it hurts a lot more than usual.",
    "I’m not sure what happened, but I have this really bad ache in my back that’s making it hard to sleep.",
    "I was gardening yesterday, and now my back is really sore; it feels like I pulled something.",
    "I have this nagging pain in my lower back that started after I sat at my desk for too long.",
    "My back has been throbbing since I went for a long walk; I didn't think it would hurt this much.",
    "I feel a sharp pain in my back when I twist or turn; it’s really uncomfortable.",
    "I think I might have strained my back; it hurts a lot when I try to lift anything.",
    "I woke up with a really bad backache, and it’s been bothering me all day.",
    "I’ve been feeling a lot of pressure in my back, and it’s making it hard to breathe comfortably.",
    "After playing sports last weekend, I’ve had this persistent pain in my back that won’t go away.",
    "I have this dull ache in my back that gets worse when I sit for too long; it’s really frustrating.",
    "I think I might have overdone it at the gym; my back is really hurting right now.",
    "I was lifting weights at the gym, and now I can’t shake this pain in my lower back.",
    "I’ve been feeling this sharp pain in my back for a couple of days, and it’s not getting any better.",
    "I don’t know what happened, but I can barely move without feeling this intense pain in my back.",
    "I’ve been having this weird pain in my back that feels like it’s radiating down my legs.",
    "I’ve been experiencing a lot of discomfort in my back lately, especially when I try to sleep.",
    "I was playing with my kids and suddenly felt a jolt in my back; now it really hurts.",
    "I’ve had this nagging pain in my back since I started my new job, and it’s really distracting.",
    "My back feels like it’s on fire; I can’t find a comfortable position to sit or lie down.",
    "I think I might have a pinched nerve or something; my back is really painful and tingly.",
    "I’ve been feeling this constant ache in my back, and it’s making it hard to focus on anything else.",
    "I woke up with this awful pain in my back, and I can’t remember doing anything to hurt it.",
    "I’ve been experiencing this sharp, shooting pain in my back that comes and goes unexpectedly.",
    "My back feels stiff and sore, especially after sitting in the car for a long time.",
    "I’ve been having this persistent pain in my back that feels worse when I cough or sneeze.",
    "I’m not sure what’s going on, but my back is really bothering me, and I need some relief.",
    "I was bending down to pick something up, and now my back is in excruciating pain.",
    "I’ve been feeling this tightness in my back for a few days, and it’s starting to worry me.",
    "I have this dull pain in my lower back that makes it hard to get out of bed in the morning.",
    "I think I might have strained my back while doing some housework; it hurts a lot.",
    "I’ve never had back pain before, but now I can’t seem to find a comfortable position.",
    "I was helping a friend move, and now I have this sharp pain in my back that won’t go away.",
    "I’ve been sitting at my desk for hours, and now my back is really stiff and painful.",
    "I have this weird pain in my back that feels like it’s radiating to my hips; it’s really uncomfortable.",
    "I’ve been feeling this intense pressure in my back, and it’s making it hard to breathe deeply.",
    "I think I might have pulled a muscle in my back; it hurts a lot when I try to stretch.",
    "I was playing basketball and landed awkwardly; now my back is really hurting.",
    "I’ve been having this constant ache in my back that seems to get worse as the day goes on.",
    "I don’t know what’s causing it, but my back has been feeling really tight and painful.",
    "I’ve been experiencing this sharp pain in my back whenever I try to lift something heavy.",
    "I think I might need to see a specialist; my back pain is really affecting my daily life.",
    "I was gardening and now my back is really sore; it feels like I overdid it.",
    "I’ve been feeling this nagging pain in my back for a week now, and it’s not improving.",
    "My back is really stiff, and I can feel a sharp pain when I try to bend over.",
    "I think I might have a slipped disc; my back hurts so much I can hardly walk.",
    "I’ve been having this dull ache in my back that gets worse when I’m stressed.",
    "I was lifting something heavy at work, and now my back is in so much pain I can hardly move.",
    "I’ve been feeling this sharp pain in my back for a few days, and it’s starting to worry me.",
    "My back feels really tight, and I can’t seem to find a comfortable position to sit in.",
    "I think I might have injured my back while exercising; it hurts a lot when I try to stretch.",
]

In [19]:
# generate random pairs from lower_back_sentences
import random

lower_back_sentence_pairs = []
for i in range(10):
    lower_back_sentence_pairs.append(random.sample(lower_back_sentences, 2))

In [21]:
# tuple to lists
contexts, claims = zip(*lower_back_sentence_pairs)

scores = scorer.score(contexts=contexts, claims=claims)

for (s1, s2), score in zip(lower_back_sentence_pairs, scores):
    print(f"{s1} - {s2}\n\t{score}")

Evaluating: 100%|██████████| 10/10 [00:08<00:00,  1.19it/s]

I feel a sharp pain in my back when I twist or turn; it’s really uncomfortable. - I was helping a friend move, and now I have this sharp pain in my back that won’t go away.
	0.0009784804424270988
I think I might have injured my back while exercising; it hurts a lot when I try to stretch. - I’ve been feeling this sharp pain in my back for a couple of days, and it’s not getting any better.
	0.0014477850636467338
I’ve been having this weird pain in my back that feels like it’s radiating down my legs. - I think I might have pulled a muscle in my back; it hurts a lot when I try to stretch.
	0.003504588734358549
I’ve been experiencing a lot of discomfort in my back lately, especially when I try to sleep. - I’ve been having this constant ache in my back that seems to get worse as the day goes on.
	0.05697380378842354
My back feels like it’s on fire; I can’t find a comfortable position to sit or lie down. - My back feels stiff and sore, especially after sitting in the car for a long time.
	0.0

Expectation:
1. Lowest, potentially harmful (relevant, **contradictory**)
2. Mid-low (less relevant, not factual)
3. High (relevant, missing facts)
4. Low, but we don't care as it is harmless (irrelevant)
5. Highest (relevant and factual)

In [27]:
# tuple to lists
contexts, claims = zip(*sentence_pairs)

scores = scorer.score(contexts=contexts, claims=claims)

Evaluating: 100%|██████████| 7/7 [00:05<00:00,  1.21it/s]


In [28]:
for (s1, s2), score in zip(sentence_pairs, scores):
    print(f"{s1} - {s2}\n\t{score}")

John is going to New York - John is heading to New York
	0.9942428469657898
John is going to New York - John is going to Brooklyn
	0.0030380827374756336
John is going to New York - John is coming from New York
	0.305054247379303
This is a fact that is happening now: John is going to New York - This is a fact that is happening now: John is coming from New York
	0.6235038638114929
John is going to New York - John is not going to New York
	0.0017278407467529178
John is going to New York - John is going to Boston
	0.002995564602315426
John is going to New York - Mary is going to New York
	0.004358239006251097


In [25]:
scores_large = scorer_large.score(contexts=contexts, claims=claims)

Evaluating: 100%|██████████| 6/6 [00:12<00:00,  2.11s/it]


In [17]:
for (s1, s2), score in zip(sentence_pairs, scores_large):
    print(f"{s1} - {s2}\n\t{score}")

John is going to NY - John is heading to NY
	0.9958599209785461
John is going to NY - John is going to Brooklyn
	0.006489399820566177
John is going to NY - John is coming from NY
	0.016506584361195564
John is going to NY - John is not going to NY
	0.0034678166266530752
John is going to NY - John is going to Boston
	0.0012338384985923767
John is going to NY - Mary is going to NY
	0.00253722514025867


1. The patient is a 36 year old male presenting with CHD*. - The patient complains and denies any previous heart problems.

	0.0013911952264606953
	(mid high, ok)
2. The patient is a 36 year old male presenting with CHD. - The patient complains of lower back pain after mild physical activity.

	0.007403689902275801
	(mid low, ok)
3. The patient is a 36 year old male presenting with CHD. - The patient presents with mild chest pain radiating to the left arm.

	0.014418686740100384
	(high, ok)
4. The patient suffers from shortness of breath after daily exercise. - The vase was on the edge of the nightstand.

	0.003466093447059393
	(should be lowest)
5. The patient suffers from shortness of breath after mild exercise. - The patient reports from dyspnea after mild physical activity.

	0.4007830023765564
	(highest, ok)

- Sentence extraction
- gliner for summary and source sentence matching? How else can we use gliner?
- ACI bench
- How can we emphasise that we found contradictory information